In [ ]:
%pip install openai --upgrade

In [ ]:
import json
with open("../dataset/rag_truth_test.json", "r") as f:
    test_data = json.load(f)

In [ ]:
test_data[0]

{'ref': 'The FBI charged a Philadelphia woman on Thursday with trying to travel overseas to fight for ISIS. She\'s one of three women arrested this week on terror charges. Two New York women were also taken into custody. An FBI complaint cites numerous social media messages dating back to August 2013 that were sent by Keonna Thomas, 30, also known as "Young Lioness" and "Fatayat Al Khilafah." One Twitter message said, "If we truly knew the realities ... we all would be rushing to join our brothers in the front lines pray ALLAH accept us as shuhada [martyrs]." Another said, "When you\'re a mujahid [violent jihadi fighter] your death becomes a wedding." The FBI said Thomas purchased an electronic visa to Turkey on March 23. Turkey is known as the easiest place from which to enter Syria and join ISIS. An ISIS manual advises recruits to buy round-trip tickets to vacation spots such as Spain and then purchase tickets for their real destination once they arrive overseas, the FBI said. On Mar

In [ ]:
from dotenv import load_dotenv
import os
from openai import OpenAI

# please set your OpenAI API key in the .env file
# OPENAI_API_KEY="sk-xxxxxxx"
load_dotenv()
api_key = os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [ ]:
from tqdm import tqdm

outputs = []

for d in tqdm(test_data):
    response = client.responses.create(
        model="gpt-4o",
        input=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "input_text",
                        "text": d["ref"]
                        + "Please judge the following statement whether it includes hallucination or not based on the references above and output 1 if it includes hallucination and 0 if not. Output should be only an number (1 or 0): ",
                    },
                ],
            },
            {"role": "user", "content": [{"type": "input_text", "text": d["text"] + "Output:"}]},
        ],
        text={"format": {"type": "text"}},
        reasoning={},
        tools=[],
        temperature=1,
        max_output_tokens=2048,
        top_p=1,
        store=True,
    )
    outputs.append(response.output[0].content[0].text)

In [ ]:
answers = [d["labels"] for d in test_data]
outputs = [int(o.strip()) for o in outputs]

In [ ]:
from collections import Counter
count = Counter(outputs)

print(count)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

accuracy_score(answers, outputs), f1_score(answers, outputs), precision_score(answers, outputs), recall_score(answers, outputs)

In [ ]:
if os.path.exists("results_test.json"):
    with open("results_test.json", "r") as f:
        data = json.load(f)
else:
    data = []

len(data)

In [ ]:
for i, d in enumerate(data):
    d["gpt4o"] = outputs[i]

In [ ]:
data[0]

In [ ]:
with open("results_test.json", "w") as f:
    json.dump(data, f)